In [184]:
import pandas as pd
dataset = pd.read_csv("data_filtered_weather_elevation.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200721 entries, 0 to 200720
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   start_time            200721 non-null  object 
 1   end_time              200721 non-null  object 
 2   user_id               200721 non-null  int64  
 3   start_latitude        200721 non-null  float64
 4   start_longitude       200721 non-null  float64
 5   end_latitude          200721 non-null  float64
 6   end_longitude         200721 non-null  float64
 7   company               200721 non-null  object 
 8   duration_min          200721 non-null  float64
 9   time-key              200721 non-null  object 
 10  time                  200721 non-null  object 
 11  temperature_2m (°C)   200721 non-null  float64
 12  rain (mm)             200721 non-null  float64
 13  snowfall (cm)         200721 non-null  float64
 14  windspeed_10m (km/h)  200721 non-null  float64
 15  

In [185]:
missing_values_adr = dataset[(dataset['start_address'].isna())|(dataset['end_address'].isna())]
missing_values_adr.to_csv("missing_address.csv")

In [186]:
cond = dataset['start_address'].isna()



        start_latitude  start_longitude  start_location
62872        49.209796        16.608238             NaN
63227        49.180849        16.604650             NaN
75951        49.209147        16.599858             NaN
77790        49.213307        16.619464             NaN
81589        49.209107        16.591822             NaN
84037        49.197796        16.594439             NaN
86701        49.197747        16.608889             NaN
92745        49.193156        16.565627             NaN
98222        49.187084        16.613401             NaN
102899       49.209716        16.635812             NaN
110932       49.214404        16.570344             NaN
127110       49.204151        16.610508             NaN
127328       49.209911        16.592478             NaN
129683       49.211111        16.623243             NaN
159433       49.185987        16.621250             NaN
164016       49.186751        16.602392             NaN
164429       49.212671        16.624424         

In [187]:
from geopy.distance import geodesic

stations_all = pd.read_csv("10e_stations_ad_el.csv")
stations_all['station_index'] = stations_all.index
stations = stations_all.to_dict(orient='records')

def find_start_location(row):
    min_distance = float('inf')
    start_location = None
    for station in stations:
        distance = geodesic((row['start_latitude'], row['start_longitude']),
                           (station['latitude'], station['longitude'])).kilometers
        if distance < min_distance:
            min_distance = distance
            if min_distance < 0.060:
                start_location = station['station_index']
            else:
                start_location = "N/A"
                
    return start_location

def find_end_location(row):
    min_distance = float('inf')
    end_location = None
    for station in stations:
        distance = geodesic((row['end_latitude'], row['end_longitude']),
                           (station['latitude'], station['longitude'])).kilometers
        if distance < min_distance:
            min_distance = distance
            if min_distance < 0.10:
                end_location = station['station_index']
            else:
                end_location = "N/A"
                
    return end_location


# Přiřazení nejbližšího indexu lokality k GPS souřadnicím
dataset.loc[cond,["start_location"]] = dataset.loc[cond,["start_latitude","start_longitude","start_location"]].apply(find_start_location, axis=1)

print((dataset.loc[cond, ["start_latitude","start_longitude","start_location"]]))

        start_latitude  start_longitude  start_location
62872        49.209796        16.608238           921.0
63227        49.180849        16.604650           194.0
75951        49.209147        16.599858           584.0
77790        49.213307        16.619464           757.0
81589        49.209107        16.591822           587.0
84037        49.197796        16.594439          1025.0
86701        49.197747        16.608889            35.0
92745        49.193156        16.565627           728.0
98222        49.187084        16.613401           226.0
102899       49.209716        16.635812           624.0
110932       49.214404        16.570344           459.0
127110       49.204151        16.610508           694.0
127328       49.209911        16.592478           933.0
129683       49.211111        16.623243           126.0
159433       49.185987        16.621250          1003.0
164016       49.186751        16.602392            54.0
164429       49.212671        16.624424         

In [188]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200721 entries, 0 to 200720
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   start_time            200721 non-null  object 
 1   end_time              200721 non-null  object 
 2   user_id               200721 non-null  int64  
 3   start_latitude        200721 non-null  float64
 4   start_longitude       200721 non-null  float64
 5   end_latitude          200721 non-null  float64
 6   end_longitude         200721 non-null  float64
 7   company               200721 non-null  object 
 8   duration_min          200721 non-null  float64
 9   time-key              200721 non-null  object 
 10  time                  200721 non-null  object 
 11  temperature_2m (°C)   200721 non-null  float64
 12  rain (mm)             200721 non-null  float64
 13  snowfall (cm)         200721 non-null  float64
 14  windspeed_10m (km/h)  200721 non-null  float64
 15  

In [189]:
print(stations_all.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1838 entries, 0 to 1837
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   latitude       1838 non-null   float64
 1   longitude      1838 non-null   float64
 2   count          1838 non-null   int64  
 3   address        1838 non-null   object 
 4   elevation      1838 non-null   int64  
 5   station_index  1838 non-null   int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 86.3+ KB
None


In [190]:
dataset.drop(columns=["start_address", "start_elevation"], axis =1, inplace = True)


dataset = dataset.merge(stations_all[["station_index", "address", "elevation"]], left_on = "start_location", right_on = "station_index", right_index= False, how = "left")
dataset.rename(columns = {"elevation":"start_elevation","address": "start_address"}, inplace = True )

dataset.drop(columns="station_index", axis =1, inplace = True)
print(dataset.columns)


Index(['start_time', 'end_time', 'user_id', 'start_latitude',
       'start_longitude', 'end_latitude', 'end_longitude', 'company',
       'duration_min', 'time-key', 'time', 'temperature_2m (°C)', 'rain (mm)',
       'snowfall (cm)', 'windspeed_10m (km/h)', 'windgusts_10m (km/h)',
       'is_day ()', 'start_location', 'end_location', 'end_address',
       'end_elevation', 'start_address', 'start_elevation'],
      dtype='object')


In [191]:
dataset_final = dataset[['start_time', 'end_time', 'user_id', 'start_latitude',
       'start_longitude', 'end_latitude', 'end_longitude', 'company',
       'duration_min', 'time-key', 'time', 'temperature_2m (°C)', 'rain (mm)',
       'snowfall (cm)', 'windspeed_10m (km/h)', 'windgusts_10m (km/h)',
       'is_day ()', 'start_location', 'end_location', 'start_address',
       'end_address', 'start_elevation', 'end_elevation']]



### čištění odlehlých (gps) hodnot

In [192]:
dataset_final = dataset_final[(dataset_final["start_latitude"] < 49.263)&(dataset_final["end_latitude"] < 49.263)&(dataset_final["start_latitude"] > 49.116)&(dataset_final["end_latitude"] > 49.116)]

dataset_final = dataset_final[(dataset_final["start_longitude"] < 16.720)&(dataset_final["end_longitude"] < 16.720)&(dataset_final["start_longitude"] > 16.485)&(dataset_final["end_longitude"] > 16.485)]

print(dataset_final)

                 start_time             end_time    user_id  start_latitude  \
0       2022-05-01 00:10:00  2022-05-01 00:16:00  321514795       49.207780   
1       2022-05-09 14:45:00  2022-05-09 14:51:00  321514795       49.207780   
2       2022-05-26 14:24:00  2022-05-26 14:30:00  321514795       49.207780   
3       2022-07-24 00:57:00  2022-07-24 01:05:00    8256880       49.207780   
4       2022-09-03 20:11:00  2022-09-03 20:28:00    8146768       49.207780   
...                     ...                  ...        ...             ...   
200716  2023-05-25 01:34:00  2023-05-25 01:46:00     375749       49.197051   
200717  2023-05-29 03:08:00  2023-05-29 03:34:00     390769       49.197445   
200718  2023-05-29 03:08:00  2023-05-29 03:33:00     470851       49.197445   
200719  2023-09-04 03:57:00  2023-09-04 04:02:00     172510       49.199359   
200720  2023-09-06 01:03:00  2023-09-06 01:14:00     166389       49.222809   

        start_longitude  end_latitude  end_longitud

### přidání elevation difference


In [193]:
dataset_final["elevation_difference"] = dataset_final["end_elevation"] - dataset_final["start_elevation"]

In [197]:
dataset_final["start_location"] = dataset_final["start_location"].astype(int)
dataset_final["start_elevation"] = dataset_final["start_elevation"].astype(int)

In [198]:
dataset_final.to_csv("23-10-29_data_filtered_weather_elevation.csv", index = False)

In [199]:
dataset_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 198945 entries, 0 to 200720
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   start_time            198945 non-null  object 
 1   end_time              198945 non-null  object 
 2   user_id               198945 non-null  int64  
 3   start_latitude        198945 non-null  float64
 4   start_longitude       198945 non-null  float64
 5   end_latitude          198945 non-null  float64
 6   end_longitude         198945 non-null  float64
 7   company               198945 non-null  object 
 8   duration_min          198945 non-null  float64
 9   time-key              198945 non-null  object 
 10  time                  198945 non-null  object 
 11  temperature_2m (°C)   198945 non-null  float64
 12  rain (mm)             198945 non-null  float64
 13  snowfall (cm)         198945 non-null  float64
 14  windspeed_10m (km/h)  198945 non-null  float64
 15  windg